# Fiddler Quick Start Guide

This guide will walk you through the basic onboarding steps required to use Fiddler for production model monitoring and explainability. API documentation can be found [here](https://docs.fiddler.ai/api-reference/python-package/)

# Step One: Client Setup

Here we will install the [Fiddler Python package](https://pypi.org/project/fiddler-client/) and establish an API connection to our Fiddler instance.

This Python client is a powerful way to:
- Upload the dataset and model to Fiddler
- Ingest production events to Fiddler

This can be done from a Jupyter Notebook or any python editor that you use to load data and build models.

<img src="images/fiddler_client.png" width=600 height=600 />

First, we need to initialize the client object by specifying:
- The `url`: url is the fiddler URL that you have been provided to access. Usually of the form ‘XXXXX.fiddler.ai’. Contact us if you don’t have it
- The `org_id`: organization id is an identifier for the account. See Fiddler_URL/settings/general to find this id (listed as "Organization ID")
<img src="images/org_id.png" width=800 height=800 />
- The `auth_token`: this token is used to authenticate access. See Fiddler_URL/settings/credentials to find, create, or change this token
<img src="images/auth_token.png" width=800 height=800 />

You can also save this config as a file called `fiddler.ini` in the same folder as the notebook/script. That saves you from specifying the parameters in every notebook and script.
<img src="images/fiddler_ini.png" width=800 height=800 />


In [1]:
!pip install fiddler-client==0.6.8;

In [2]:
%%writefile fiddler.ini

[FIDDLER]
url = http://host.docker.internal:4100
org_id = onebox
auth_token = 1rkqdQ56CKsU-QPrnhFqQz8EHbb7kpht1XxPj8RdOMw

Overwriting fiddler.ini


In [3]:
import fiddler as fdl

# client = fdl.FiddlerApi(url=url, org_id=org_id, auth_token=auth_token)
client = fdl.FiddlerApi()

Fiddler has three primary constructs, namely projects, datasets and models. This diagram illustrates the relationship between the three.
<img src="images/projects_data_models.png" width=600 height=600 />

The Fiddler client provides a number of methods. API documentation can be found [here](https://docs.fiddler.ai/api-reference/python-package/)


# Step Two: Create Project

Here we will create a project, a convenient container for housing the models and datasets associated with a given ML use case.

For the purposes of a full quick start, it is best to create a `project_id` with a unique name to best track your progress.

In [4]:
project_id = 'tutorial'

In [5]:
# Creating our project using project_id
if project_id not in client.list_projects():
    client.create_project(project_id)

# Step Three: Upload Baseline Data

Here we will upload the datasets that will serve as baselines for various product capabilities, including monitoring of model performance, prediction & feature drift, and data errors; generating prediction-level (point) and model-level (global) explanations; and calculating various bias metrics.

We recommend using the model's training and test set for the most faithful and actionable metrics. In addition to the model's features and labels, Fiddler requires a few additional attributes to unlock its full suite of capabilities:

*   Model predictions (Mandatory: serves as a baseline for prediction drift)
*   Model decisions* (Optional: used to monitor model decsions over time, e.g. loan approved vs denied. The data uploaded initially can be random)
*  Model metadata* (Any additional fields relevant for model analysis. In the event you intend to use Fiddler to detect model bias, include any relevant protected attributes here, e.g. gender, race, age)

## Load dataset

Load the data you are going to use for training your model. For this tutorial, we will be using an auto insurance dataset that can be found [here](https://www.kaggle.com/somjee/auto-insurance-customerlifetimevalue?select=data.csv).

In [6]:
import pandas as pd

# https://www.kaggle.com/somjee/auto-insurance-customerlifetimevalue?select=data.csv
df = pd.read_csv('/app/fiddler_samples/samples/datasets/auto_insurance/data.csv')
df = df.rename(columns={"State": "Location State"})
df.columns = [x.lower().replace(' ', '_') for x in df.columns]
df.head()

,customer,location_state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


## Split Dataset into Train/Test

Now we will split our dataset into a train/test set to be used in training our model.

In [7]:
# !pip install scikit-learn==0.21.2

In [8]:
from sklearn.model_selection import train_test_split

df_train = df.sample(frac=0.8,random_state=200)
df_test = df.drop(df_train.index)

## Upload dataset

To upload a model, you first need to upload a sample of the data of the model’s inputs, targets, and additional metadata that might be useful for model analysis. This data sample helps us (among other things) to infer the model schema and the data types and values range of each feature.
- This sample has to be a flat table that can be loaded as a pandas DF (```upload_dataset()```).
- This input data sample is used for many downstream functions in Fiddler, including: shapley value methods, what-if (ICE) plots, PDP plots, drift, outliers, and data integrity.
- We suggest uploading a sample of the model’s training data as it’s the most meaningful for the tasks listed above. For example, model outliers should be ideally based on the training data as that’s the data the model has seen. 
- You can upload multiple datasets with string identifiers, but we currently do not ascribe any meaning to those. For example: ```dataset={'data': df}``` or ```dataset={'train': train_df, 'test': test_df}```.
- Currently we support two input types:
    - Tabular
    - Single string text, meaning text data in a single column

In [9]:
dataset_id = 'auto_insurance'
dataset_id

'auto_insurance'

Now, we will create a schema for our dataset, and upload the dataset to Fiddler.

For the purposes of this tutorial, we have already used the model we will be training in a later step to create **predictions** for our model. These predictions will be imported from a separate file, and appended as a column to this dataset. Fiddler utilizes these predictions for **TODO**.

In [10]:
if dataset_id not in client.list_datasets():
    df_schema = fdl.DatasetInfo.from_dataframe(df, max_inferred_cardinality=1000)
    upload_result = client.upload_dataset(
        dataset={'train': df_train,
                 'test': df_test},
        dataset_id=dataset_id,
        info=df_schema)
else:
    df_schema = client.get_dataset_info(dataset_id)

df_schema

DatasetInfo:
  display_name: 
  files: []
  columns:
                               column     dtype count(possible_values)  \
    0                        customer    STRING                          
    1                  location_state  CATEGORY                      5   
    2         customer_lifetime_value     FLOAT                          
    3                        response  CATEGORY                      2   
    4                        coverage  CATEGORY                      3   
    5                       education  CATEGORY                      5   
    6               effective_to_date  CATEGORY                     59   
    7                employmentstatus  CATEGORY                      5   
    8                          gender  CATEGORY                      2   
    9                          income   INTEGER                          
    10                  location_code  CATEGORY                      3   
    11                 marital_status  CATEGORY            

# Step Four: Create and Train a Model

## Create Model Schema

As you must have noted, in the dataset upload step we did not ask for the model’s features and targets, or any model specific information. That’s because we allow for linking multiple models to a given dataset schema. Hence we require an Infer model schema step which helps us know the features relevant to the model and the model task. Here you can specify the input features, the target column, decision columns and metadata columns, and also the type of model.
- We can infer the model task from the target column, or it can explicitly set. Currently we support three model types:
    - Regression
    - Binary Classification
    - Multi-class Classification

In [11]:
target = 'customer_lifetime_value'
continuous_features = ['income', 'monthly_premium_auto', 'months_since_last_claim', 'months_since_policy_inception',
                        'number_of_open_complaints', 'number_of_policies', 'total_claim_amount']
categorical_features = ['location_state', 'employmentstatus', 'policy_type', 'policy', 'vehicle_class','vehicle_size']

feature_columns = list(continuous_features + categorical_features)
metadata_cols = ['gender']

model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=client.get_dataset_info(dataset_id),
    target=target, 
    features=feature_columns,
    metadata_cols=metadata_cols,
    display_name='Gradient Boosting Regressor',
    description='this is a GradientBoostingRegressor model from the tutorial',
)

model_info

ModelInfo:
  display_name: Gradient Boosting Regressor
  description: this is a GradientBoostingRegressor model from the tutorial
  input_type: ModelInputType.TABULAR
  model_task: ModelTask.REGRESSION
  inputs:
                               column     dtype count(possible_values)  \
    0                  location_state  CATEGORY                      5   
    1                employmentstatus  CATEGORY                      5   
    2                          income   INTEGER                          
    3            monthly_premium_auto   INTEGER                          
    4         months_since_last_claim   INTEGER                          
    5   months_since_policy_inception   INTEGER                          
    6       number_of_open_complaints   INTEGER                          
    7              number_of_policies   INTEGER                          
    8                     policy_type  CATEGORY                      3   
    9                          policy  CATEGORY 

## Train model

Build and train your model. For this model, we will be creating a Pipeline that will transform the data passed in, and then run that data through a gradient boosting regressor.

In [12]:
# references
# https://scikit-learn.org/stable/modules/preprocessing.html
# https://songxia-sophia.medium.com/two-machine-learning-algorithms-to-predict-xgboost-neural-network-with-entity-embedding-caac68717dea
# https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.pipeline
from sklearn.compose import ColumnTransformer

target = 'customer_lifetime_value'
continuous_features = ['income', 'monthly_premium_auto', 'months_since_last_claim', 'months_since_policy_inception',
                        'number_of_open_complaints', 'number_of_policies', 'total_claim_amount']
categorical_features = ['location_state', 'employmentstatus', 'policy_type', 'policy', 'vehicle_class','vehicle_size']

category_transformer = sklearn.pipeline.Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[('cat', category_transformer, categorical_features)])

model = GradientBoostingRegressor(learning_rate=0.1,
                                  n_estimators=100,
                                  max_depth=7)
model_pipeline = sklearn.pipeline.Pipeline(steps=[('preprocessor', preprocessor),
                                                  ('model', model)])

model_pipeline.fit(df_train.loc[:, df_train.columns !=  target], df_train[target])

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('onehot',
                                                                   OneHotEncoder(categorical_features=None,
                                                                                 categories=None,
                                                                                 drop=None,
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='ignore',
                                       

# Step Five: Create Model Directory

Next step, we need to save the model and any pre-processing step you had on the input features (for example Categorical encoder, Tokenization, ...).  
We currently support the following stored model formats:
- For sklearn API based models, pickled models, or any storage format that you can load in the package.py (details below).
- For TF, we support TF Saved Model and Keras .h5   

Note:
- Keras models have to have their input tensor differentiable if Integrated Gradients support is desired
- We also need to save the data preprocessing pipeline code, if any. This will be accessed in the package.py

In total, we will have a `model.yaml`, a `model.pkl`, and a `package.py` file within our model directory.

In [13]:
# Create model directory
import pathlib
import shutil

model_id = 'tutorial_gradient_boosting_regressor'
model_dir = pathlib.Path(model_id)
shutil.rmtree(model_dir, ignore_errors=True)
model_dir.mkdir()

## Save model and schema (`model.pkl` and `model.yaml`)

In [14]:
# Write model schema file to model directory
import yaml
import pickle

# Saving model
with open(model_dir / 'model.pkl', 'wb') as pkl_file:
    pickle.dump(model_pipeline, pkl_file)

# Saving schema
with open(model_dir / 'model.yaml', 'w') as yaml_file:
    yaml.dump({'model': model_info.to_dict()}, yaml_file)

## Create `package.py`

`package.py` is a Python module that

- Facilitates model loading
- Implements interfaces necessary for the Fiddler platform to interact with models.

This provides the flexibility to enable Fiddler to support a wide variety of complex models. More information can be found [here](https://docs.fiddler.ai/api-reference/package-py/)

In [15]:
%%writefile tutorial_gradient_boosting_regressor/package.py

import pandas as pd
from pathlib import Path
import pickle as pkl
import os

PACKAGE_PATH = Path(__file__).parent
TARGET = 'customer_lifetime_value'
PREDICTION = 'predicted_customer_lifetime_value'

class GBRegressor:
    """A Gradient Boosting Regressor predictor for auto_insurance data.
       This loads the predictor once and runs for each call to predict.
    """

    def __init__(self, model_path, output_column=None):
        """
        :param model_path: The directory where the model is saved.
        :param output_column: list of column name(s) for the output.
        """
        self.model_path = model_path
        self.output_column = output_column

        file_path = os.path.join(self.model_path, 'model.pkl')
        with open(file_path, 'rb') as file:
            self.model = pkl.load(file)

    def predict(self, input_df):
        return pd.DataFrame(
            self.model.predict(input_df.loc[:, input_df.columns != TARGET]), 
            columns=self.output_column
        )

def get_model():
    return GBRegressor(model_path=PACKAGE_PATH, output_column=[PREDICTION])

Writing tutorial_gradient_boosting_regressor/package.py


# Step Six: Upload Model

## Validate Model Package
 
This step finds issues with the `package.py` composed above to enable easy debugging.

In [16]:
from fiddler import PackageValidator

validator = PackageValidator(model_info, df_schema, model_dir)
passed, errors = validator.run_chain()

Validation Result: PASS


## Upload Model

Now that we have all the parts that we need, we can go ahead and upload the model to the Fiddler platform. You can use the [upload_model_package](https://docs.fiddler.ai/api-reference/python-package/#upload-model-package) to upload this entire directory in one shot. We need the following for uploading a model:
- The `path` to the directory
- The `project_id` to which the model belongs
- The `model_id`, which is the name you want to give the model. You can access it in Fiddler henceforth via this ID
- The `dataset` which the model is linked to (optional)  

In [17]:
# Let's first delete the model if it already exists in the project
if model_id in client.list_models(project_id):
    client.delete_model(project_id, model_id)
    print('Model deleted')
    
client.upload_model_package(artifact_path=model_dir, project_id=project_id, model_id=model_id)
f"Project '{project_id} now contains model '{model_id}'"

"Project 'tutorials now contains model 'tutorial_gradient_boosting_regressor'"

Let's trigger the model predictions by calling the following function:

In [22]:
client.trigger_model_predictions(project_id, model_id, dataset_id)

'Successfully processed and uploaded predictions for dataset auto_insurance with model tutorial_gradient_boosting_regressor'

## Run model

Now, let's test out our model by interfacing with the client and calling [run model](https://docs.fiddler.ai/api-reference/python-package/#run-model).

In [21]:
prediction_input = df_train[0: 10]
result = client.run_model(project_id, model_id, prediction_input, log_events=True)
result

,predicted_customer_lifetime_value
0,6353.814232
1,6935.858778
2,4736.136645
3,6399.515196
4,11618.909563
5,6902.586371
6,8120.247485
7,8618.783066
8,6260.395796
9,19947.630198


# Step Seven: Simulate Monitoring Traffic

In this step, we will be simulating traffic to send for our model monitoring by using [publish_event](https://docs.fiddler.ai/api-reference/python-package/#publish-event). This will be the equivalent of running our model separately on some data, and either sending to Fiddler then, or saving this information to a log and sending at a later point.

For this demonstration, we will be going with a log-related approach. This log will contain rows that have inputs, predictions, and real targets. To most accurately simulate this as a time-series event, we will also be calling a function to generate a timestamp in the last 2 weeks. Real data will ideally have a timestamp related to when the event took place; otherwise, the current time will be used.

**Note**: The timestamp must be in UTC milliseconds. See [here](https://docs.fiddler.ai/api-reference/python-package/#publish-event) for more details

In [23]:
import datetime
import time
from IPython.display import clear_output
from random import sample, randint
NUM_EVENTS_TO_SEND = 50

def getTimestampFromPastTwoWeeks():
    """
    Generate a randomized timestamp from the past two weeks. Timestamp is in 
    milliseconds since epoch in UTC.
    """
    TWO_WEEKS_MS = 604800 * 2 * 1000
    current_time_in_ms = round(time.time() * 1000)
    
    random_time_in_past_two_weeks = current_time_in_ms - randint(0, TWO_WEEKS_MS)
    return random_time_in_past_two_weeks

        
event_log = pd.read_csv('/app/fiddler_samples/samples/datasets/auto_insurance/event_log.csv')

# Convert this dataframe into a list of dictionary events, where each event is its own dictionary
event_list_dict = event_log.sample(n=NUM_EVENTS_TO_SEND).to_dict(orient='records') 

for ind, event_dict in enumerate(event_list_dict):
    event_ms_time_stamp = getTimestampFromPastTwoWeeks()
    result = client.publish_event(project_id, model_id, event_dict, event_time_stamp=event_ms_time_stamp)
    
    clear_output(wait = True)
    readable_timestamp = datetime.datetime.fromtimestamp(event_ms_time_stamp/1000.0)
    
    print(f'Sending {ind+1} / {NUM_EVENTS_TO_SEND} \n{readable_timestamp} UTC: \n{event_dict}')
    time.sleep(1)
    

Sending 50 / 50 
2020-12-14 19:38:55.172000 UTC: 
{'customer': 'DE55857', 'location_state': 'Washington', 'customer_lifetime_value': 2465.444865, 'response': 'No', 'coverage': 'Basic', 'education': 'Bachelor', 'effective_to_date': '1/30/11', 'employmentstatus': 'Employed', 'gender': 'F', 'income': 64997, 'location_code': 'Suburban', 'marital_status': 'Single', 'monthly_premium_auto': 63, 'months_since_last_claim': 7, 'months_since_policy_inception': 99, 'number_of_open_complaints': 1, 'number_of_policies': 1, 'policy_type': 'Personal Auto', 'policy': 'Personal L3', 'renew_offer_type': 'Offer2', 'sales_channel': 'Agent', 'total_claim_amount': 383.442328, 'vehicle_class': 'Four-Door Car', 'vehicle_size': 'Small', 'predicted_customer_lifetime_value': 6956.451807764521, '__event_type': 'execution_event', '__occurred_at': 1607974735172}
